In [1]:
from fastai.vision import *
import onnx
import onnxruntime

In [2]:
learn = load_learner('.')
fastai_model = learn.model.eval()

In [3]:
fastai_model[1]

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=4096, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=True)
)

In [4]:
imagenet_mean, imagenet_std = torch.tensor([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])

In [5]:
class ImageScale(nn.Module):
    def forward(self, x):
        xx = (x/255.0-imagenet_mean[...,None,None]) / imagenet_std[...,None,None] 
        return xx.unsqueeze(0)
    
scale_layer = ImageScale()
softmax_layer = torch.nn.Softmax(dim=1)

In [6]:
final_model = nn.Sequential(scale_layer, fastai_model, softmax_layer)

In [7]:
fastai_img = open_image('normal.jpeg').data * 255.0
with torch.no_grad():
    print(final_model(fastai_img))

tensor([[0.0028, 0.9972]])


In [8]:
fastai_model[1]

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=4096, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=2, bias=True)
)

In [9]:
class Reshape(nn.Module):
    def forward(self, x):
        xx = torch.reshape(x, [-1, 4096])
        return xx

fastai_model[1][1] = Reshape()

In [10]:
fastai_img = open_image('normal.jpeg').data * 255.0
with torch.no_grad():
    print(final_model(fastai_img))

tensor([[0.0028, 0.9972]])


In [11]:
def preprocess(img, sz):
    _, r, c = img.shape
    ratio = min(r/sz, c/sz)
    r, c = int(round(r/ratio)), int(round(c/ratio))
    img = img.resize((3, r, c)).data
    
    row = int((r-sz+1)*0.5)
    col = int((c-sz+1)*0.5)
    img = img[:, row:row+sz, col:col+sz]    
    return img

In [12]:
fastai_img = preprocess(open_image('normal.jpeg'), 256) * 255.0
with torch.no_grad():
    print(final_model(fastai_img))

tensor([[1.0000e+00, 8.8745e-07]])


In [13]:
model_name = 'pneumonia.onnx'

In [14]:
dummy_image = torch.autograd.Variable(torch.randn(3, 256, 256))
torch.onnx.export(final_model, dummy_image,
                  model_name, input_names = ['image'], output_names = ['diagnosis'])
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

In [15]:
session = onnxruntime.InferenceSession(model_name)

In [16]:
fastai_img = preprocess(open_image('normal.jpeg'), 256) * 255.0
img_data = fastai_img.numpy()
img_data.shape

(3, 256, 256)

In [17]:
onnx_result = session.run([], {'image': img_data})
onnx_result

[array([[9.999992e-01, 8.874645e-07]], dtype=float32)]